### Setup para o Google Colab

In [1]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/")
os.getcwd()

'/content/drive/MyDrive'

In [ ]:
! git clone https://github.com/mateuslatrova/correcao-enem.git

Cloning into 'correcao-enem'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 119 (delta 48), reused 114 (delta 47), pack-reused 0
Receiving objects: 100% (119/119), 1.77 MiB | 4.29 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
os.chdir("/content/drive/MyDrive/correcao-enem")
os.getcwd()

'/content/drive/MyDrive/correcao-enem'

In [ ]:
! git pull

In [ ]:
# requirements on Google Colab:
!pip install scipy scikit-learn datasets evaluate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.3 MB/s eta 0:00:00


### Imports

In [2]:
import os
while os.path.basename(os.getcwd()) != "correcao-enem":
    os.chdir("..")

In [4]:
from correction.config_reader.config_reader import ConfigReader
from correction.topic_deviation.combinator import EssayAndPromptDatasetsCombinator
from correction.topic_deviation.essay_preprocessor import EssayDatasetPreprocessorForTopicDeviation
from correction.topic_deviation.prompt_preprocessor import PromptDatasetPreprocessorForTopicDeviation
from correction.topic_deviation.fine_tuner import TopicDeviationFineTuner


from datasets import (
    load_dataset,
    load_from_disk,
)
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer

### Variáveis globais

In [5]:
config_path = "correction/topic_deviation/config.yaml"
config_reader = ConfigReader(config_path)

## Exploração do dataset de redações

### Carregamento e primeiro contato com os dados

In [7]:
raw_essay_dataset_filepath = config_reader.get_dataset_path(
    level=config_reader.raw_files_dirname, 
    filename=config_reader.get("raw_essay_filename"), 
    file_extension=config_reader.get("raw_files_extension")
)
dataset = load_dataset("csv", data_files=raw_essay_dataset_filepath)

dataset = dataset["train"]
dataset.set_format("pandas")
df = dataset[:]
df.head()

,id,id_prompt,title,essay,grades,final_grade,is_ENEM,is_convertible,general,specific
0,0,0,O estudo é a base para melhorar de vida,['Com os avanços que a sociedade deu em tecnol...,"[160, 200, 200, 160, 160]",880,True,True,"Texto muito bom, um pouco prejudicado pelos as...",['1) Há um certo número de problemas de lingua...
1,1,0,O mal do século e o combate através da fé,"['De acordo com pesquisas, o Brasil é o país d...","[120, 120, 120, 160, 120]",640,True,True,Diz um linguista que comunicação é o que o lei...,['1) O texto não apresenta problemas graves de...
2,2,0,Em uma pesquisa recente,['Em uma pesquisa recente realizada pela ONG O...,"[120, 80, 120, 80, 120]",520,True,True,"Texto fraco, que apenas tangencia o tema, dedi...",['1) O autor se expressa com dificuldade. Bast...
3,3,0,O conhecimento liberta,"['A ideia de vencer na vida, presente no imagi...","[160, 200, 160, 200, 160]",880,True,True,"Texto muito bom, com pequenos problemas menore...","['1) Texto muito bem escrito, com alguns probl..."
4,4,0,O caminho do sucesso,"['O caminho do sucesso', 'É de fundamental imp...","[120, 80, 40, 40, 80]",360,False,True,"Lamentavelmente, o texto é muito fraco, em esp...",['1) O autor tem uma dificuldade de se express...


### Validando o formato das redações

In [6]:
samples = dataset.shuffle(7).select(range(3))
samples.set_format("pandas")
samples_df = samples[:]
essay_examples = samples_df["essay"].tolist()
essay_examples

["['O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa.', 'Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa.', 'Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais.', 'Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais.', 'Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de m

In [7]:
# Como há apenas valores "True" nessa máscara, todas as redações iniciam com '[' e terminam com ']'.
mask = (df["essay"].str.startswith("[")) & (
    df["essay"].str.endswith("]")
)
set(mask.values.tolist())

{True}

In [8]:
# Para remover esses caracteres de início e fim:
for example in essay_examples:
    print(example.strip("['']"))

O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa.', 'Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa.', 'Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais.', 'Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais.', 'Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de mais 

In [9]:
# Além disso, aparentemente, os parágrafos da redação são iniciados e terminados pelos caracteres "\'" ou apenas "'".
# Para validar isso, faremos o seguinte teste:
for example in essay_examples:
    example = example.strip("['']")
    removed_quote_and_comma = example.replace("',", "")
    removed_quote = removed_quote_and_comma.replace("'", "")
    removed_slash = removed_quote.replace("\\", "")
    print(removed_slash)

O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa. Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa. Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais. Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais. Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de mais escolas com 

In [10]:
# Um jeito mais eficiente que descobri para realizar a mesma tarefa:
for example in essay_examples:
    essay = ""
    for paragraph in eval(example):
        essay += paragraph
    print(essay)

O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa.Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa.Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais.Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais.Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de mais escolas com qual

### Validando o que significa o campo "id_prompt"

In [11]:
# Validando se o campo "id_prompt" é de fato um identificador para o tema da redação
id_prompt_to_count = df["id_prompt"].value_counts().to_dict()
list(id_prompt_to_count.items())[0:10]

[(0, 20),
 (15, 20),
 (40, 20),
 (39, 20),
 (37, 20),
 (35, 20),
 (34, 20),
 (33, 20),
 (27, 20),
 (1, 20)]

In [12]:
# Observando o dataframe abaixo, podemos concluir que, de fato, o campo "id_prompt" é um identificador para
# o tema da redação.
df[df["id_prompt"] == 1].sample(frac=1).head(10)

,id,id_prompt,title,essay,grades,final_grade,is_ENEM,is_convertible,general,specific
34,34,1,O movimento conservador até a Física explica,"['Desde o fim da Ditadura Militar, o movimento...","[120, 120, 120, 80, 80]",520,True,True,"Texto regular, com muitos problemas de linguag...",['1) Primeiro parágrafo: a) se o autor quer fa...
22,22,1,Um novo olhar sobre o Brasil,['O nosso país está vivenciando diversos acont...,"[0, 0, 0, 0, 0]",0,True,True,"Infelizmente, a redação é fraca. No entanto, e...",['1) Primeiro parágrafo: a introdução é muito ...
31,31,1,Nada que preste,"['Na França, em 1789 à 1799 ocorreu a sua revo...","[160, 80, 40, 80, 40]",400,False,True,"Texto fraco. Infelizmente, o autor não persegu...",['1) Primeiro e segundo parágrafo: conquanto o...
36,36,1,Remédio para a falta de responsabilidade,['Os resultados das urnas nas últimas eleições...,"[200, 160, 160, 200, 120]",840,True,True,Texto muito bom. Satisfaz plenamente as compet...,[]
39,39,1,A onda democrática,"['Em 2018, o Brasil encontrou-se em uma eleiçã...","[120, 160, 120, 120, 120]",640,True,True,Texto regular. Poderia ser muito melhor se o a...,['1) Primeiro parágrafo: o autor teve dificuld...
37,37,1,Efeitos do conservadorismo,"['Desde o iluminismo, entende-se que uma socie...","[0, 80, 80, 80, 80]",320,True,True,"Infelizmente, o texto é muito fraco, principal...",['1) Primeiro parágrafo: o autor começa usando...
25,25,1,O homem é naturalmente bom,['O conservadorismo está em voga na contempora...,"[160, 160, 120, 160, 160]",760,True,True,"Texto bom, com poucos deslizes e erros de ling...",[]
32,32,1,A onda conservadora e o Brasil nos próximos anos,['O país está em transição para um novo govern...,"[40, 40, 40, 40, 40]",200,False,True,"Lamentavelmente, o texto só pode ser considera...",['1) Primeiro parágrafo: a) pela ordem dos ter...
33,33,1,O futuro é agora: politize-se!,"['O Brasil carrega desde o passado, caracterís...","[80, 80, 80, 80, 80]",400,True,True,"Lamentavelmente, o texto é muito fraco. As úni...",['1) Primeiro parágrafo: a frase em vermelho n...
23,23,1,A nova hegemonia política do Brasil,['A maioria dos países da América do Sul passa...,"[160, 160, 120, 120, 120]",680,True,True,"Texto bom, apesar da linguagem coloquial, da f...",['1) Primeiro parágrafo: a) A primeira frase t...


### Validando remoção de colunas sem dados

#### Obs.: no novo dataset, essas colunas possuem dados, então não será necessário removê-las

In [13]:
# Como todos valores dessas duas colunas são nulos, podemos removê-las.
columns_to_be_removed = ["general", "specific"]
for column in columns_to_be_removed:
    print(all(df[column].isna()))

False
False


### Validando se é possível remover valores estranhos no campo "title"

#### Obs.: no novo dataset, os valores dessa coluna já foram limpos.

In [14]:
title_to_count = df["title"].value_counts()
title_to_count = {title: count for title, count in title_to_count.items() if count > 1}
title_to_count

{}

In [15]:
# Depois de analisar todos esses títulos estranhos, concluiu-se que todos eles são títulos
# inválidos, ou são simplesmente um valor indicando que a redação não teve título.
suspicious_titles = list(title_to_count.keys())
df[df["title"].isin(suspicious_titles)].sort_values(by=["title"])

,id,id_prompt,title,essay,grades,final_grade,is_ENEM,is_convertible,general,specific


In [16]:
# Portanto, vamos apenas colocar um valor nulo para todos esses títulos inválidos
df.loc[df["title"].isin(suspicious_titles), "title"] = pd.NA
df[df["title"].isin(suspicious_titles)]

,id,id_prompt,title,essay,grades,final_grade,is_ENEM,is_convertible,general,specific


In [17]:
df["title"].unique().tolist()

['O estudo é a base para melhorar de vida',
 'O mal do século e o combate através da fé',
 'Em uma pesquisa recente',
 'O conhecimento liberta',
 'O caminho do sucesso',
 'Nem tudo é dinheiro',
 'Desejo pelo Sucesso',
 'Estudar',
 'O que realmente é importante para a melhoria de vida',
 'Fé: essencial para a vida em geral ou para vida espiritual',
 'A fé é o equilíbrio do brasileiro',
 'A fé ao encontro da ciência',
 'A arte do equilíbrio da vida',
 'Ser vencedor é uma escolha',
 'Povo que anda com fé',
 'Vencer na vida',
 'Vencer ou ser?',
 '“E essa é a vitória que vence o mundo: a nossa fé”',
 'O que é mais importante para vencer na vida?',
 'Fatores mais importantes para vencer na vida',
 'Por que uma onda conservadora?',
 'O conservadorismo no Brasil (2)',
 'Um novo olhar sobre o Brasil',
 'A nova hegemonia política do Brasil',
 'O que falta ao Brasil para prosperar?',
 'O homem é naturalmente bom',
 'O conservadorismo no Brasil',
 'A renovação do Brasil',
 'A Volta da Política Con

Porém, depois de analisar mais exemplos, percebeu-se que há mais títulos inválidos. Então, caso seja realmente necessário remover esses títulos inválidos para alguma tarefa, isso será realizado posteriormente devido ao grande número de exemplos.

### Análise do campo "grades"

In [18]:
samples = dataset.shuffle(7).select(range(3))
samples.set_format("pandas")
samples_df = samples[:]
grades_examples = samples_df["grades"].tolist()
grades_examples

['[0, 0, 0, 0, 0]', '[200, 120, 120, 120, 160]', '[120, 120, 120, 120, 120]']

In [19]:
type(grades_examples[0])

str

In [20]:
example = grades_examples[0]
grades_without_brackets = example.strip("[]")
grades_without_brackets

'0, 0, 0, 0, 0'

In [21]:
# Obtendo a nota de cada competência separadamente, podemos criar uma feature de nota por competência, sendo do tipo inteiro.
[int(grade) for grade in grades_without_brackets.split(",")]

[0, 0, 0, 0, 0]

In [22]:
# Uma forma mais eficiente de realizar a mesma tarefa acima:
for example in grades_examples:
    print(eval(example))

[0, 0, 0, 0, 0]
[200, 120, 120, 120, 160]
[120, 120, 120, 120, 120]


## Exploração do dataset de temas

In [23]:
raw_prompt_dataset_filepath = config_reader.get_dataset_path(
    level=config_reader.raw_files_dirname, 
    filename=config_reader.get("raw_prompt_filename"), 
    file_extension=config_reader.get("raw_files_extension")
)

prompt_df = pd.read_csv(raw_prompt_dataset_filepath)
prompt_df.head()

,id,source,title,prompt,supporting_text
0,0,UOL,O que é mais importante para vencer na vida?,['Reportagem publicada pelo UOL Economia no mê...,"['Pessoas valorizam fé, em vez de estudo e tra..."
1,1,UOL,A onda conservadora e o Brasil nos próximos an...,"['Em meados desta década, que se aproxima do f...","['A nova onda conservadora no Brasil', 'Após e..."
2,2,UOL,A terapia de reversão da orientação sexual,['Um assunto sensível transformou-se em notíci...,"['Os fatos', 'Uma decisão em caráter liminar d..."
3,3,UOL,Agrotóxicos ou defensivos agrícolas: dois nome...,"['O fato é recente, mas o debate já dura mais ...","['Brasil, potência agrícola', 'O Governo do pr..."
4,4,UOL,Artes e educação física: opcionais ou obrigató...,"['No mês de setembro passado, o governo federa...","['Educação integral', 'Ao justificar a reforma..."


### Entendendo a coluna "prompt" (descrição do tema)

In [24]:
topic_example = prompt_df.loc[0, "prompt"]
topic_example

"['Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. Os resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.']"

In [25]:
eval(topic_example)

['Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. Os resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.']

In [26]:
type(eval(topic_example))

list

In [27]:
topic_text = ""
for paragraph in eval(topic_example):
    if not paragraph.endswith("."):
        paragraph = paragraph + "."
    topic_text += paragraph
print(type(topic_text))
topic_text

<class 'str'>


'Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. Os resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.'

### Entendendo a coluna "supporting text" (texto motivador sobre o tema)

In [29]:
supporting_text_example = prompt_df.loc[0, "supporting_text"]
supporting_text_example

"['Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida', 'A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio finance

In [30]:
eval(supporting_text_example)

['Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida',
 'A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio finance

In [31]:
type(eval(supporting_text_example))

list

In [32]:
supporting_text = ""
for paragraph in eval(supporting_text_example):
    if not paragraph.endswith("."):
        paragraph = paragraph + "."
    supporting_text += paragraph
print(type(supporting_text))
supporting_text

<class 'str'>


'Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida.A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio financeiro d

### Entendendo a coluna "title" (título do tema)

In [33]:
title_example = prompt_df.loc[0, "title"]
print(title_example)

O que é mais importante para vencer na vida?  


In [34]:
print(type(title_example))

<class 'str'>


## Pré-processamento das redações para detecção de fuga ao tema

In [8]:
raw_essay_dataset_filepath = config_reader.get_dataset_path(
    level=config_reader.raw_files_dirname, 
    filename=config_reader.get("raw_essay_filename"), 
    file_extension=config_reader.get("raw_files_extension")
)
dataset_dict = load_dataset("csv", data_files=raw_essay_dataset_filepath)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'id_prompt', 'title', 'essay', 'grades', 'final_grade', 'is_ENEM', 'is_convertible', 'general', 'specific'],
        num_rows: 385
    })
})

In [9]:
essay_dataset = dataset_dict["train"]
essay_dataset

Dataset({
    features: ['id', 'id_prompt', 'title', 'essay', 'grades', 'final_grade', 'is_ENEM', 'is_convertible', 'general', 'specific'],
    num_rows: 385
})

In [10]:
preprocessor = EssayDatasetPreprocessorForTopicDeviation(essay_dataset)
preprocessed_dataset_dict = preprocessor.preprocess_dataset()
preprocessed_dataset_dict

Filter: 100%|██████████| 558/558 [00:00<00:00, 21147.37 examples/s]


DatasetDict({
    train: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 459
    })
    test: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 98
    })
    validation: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 99
    })
})

In [11]:
preprocessed_dataset = preprocessed_dataset_dict["train"]
preprocessed_dataset.set_format("pandas")
df = preprocessed_dataset[:]
df

,essay_id,topic_id,essay_text,label,is_artificial
0,1,0,"De acordo com pesquisas, o Brasil é o país da ...",1,False
1,2,0,Em uma pesquisa recente realizada pela ONG Oxf...,1,False
2,3,0,"A ideia de vencer na vida, presente no imaginá...",1,False
3,5,0,"Hoje em dia, vencer na vida é algo que muitos ...",1,False
4,6,0,Hoje em dia a maioria dos brasileiros veem a f...,1,False
...,...,...,...,...,...
454,12,19,A busca por oportunidades para melhorar de vid...,0,True
455,805,8,"Recentemente, vêm sendo levantados questioname...",0,True
456,159,37,A revolução transumanista é a alteração ou est...,0,True
457,678,6,"Olhando para o passado vemos que em Atenas, na...",0,True


In [12]:
# Save preprocessed dataset for later use:
preprocessed_essay_dataset_filepath = config_reader.get_dataset_path(
    level=config_reader.preprocessed_files_dirname, 
    filename=config_reader.get("essay_preprocessed_dirname"), 
)
preprocessed_dataset_dict.save_to_disk(preprocessed_essay_dataset_filepath)

Saving the dataset (1/1 shards): 100%|██████████| 99/99 [00:00<00:00, 7898.28 examples/s]


## Pré-processamento do dataset de temas para detecção de fuga ao tema

In [13]:
raw_prompt_dataset_filepath = config_reader.get_dataset_path(
    level=config_reader.raw_files_dirname, 
    filename=config_reader.get("raw_prompt_filename"), 
    file_extension=config_reader.get("raw_files_extension")
)
dataset_dict = load_dataset("csv", data_files=raw_prompt_dataset_filepath)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'title', 'prompt', 'supporting_text'],
        num_rows: 44
    })
})

In [14]:
prompt_dataset = dataset_dict["train"]
prompt_dataset

Dataset({
    features: ['id', 'source', 'title', 'prompt', 'supporting_text'],
    num_rows: 44
})

In [15]:
preprocessor = PromptDatasetPreprocessorForTopicDeviation(prompt_dataset)
preprocessed_prompt_dataset = preprocessor.preprocess_dataset()
preprocessed_prompt_dataset

Dataset({
    features: ['topic_id', 'topic_text'],
    num_rows: 44
})

In [16]:
samples = preprocessed_prompt_dataset.select([0, 5, 10])
for sample in samples:
    print(sample["topic_text"])

O que é mais importante para vencer na vida?  .Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida.A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter a

In [17]:
# Save preprocessed dataset for later use:
dataset_dict["train"] = preprocessed_prompt_dataset
preprocessed_prompt_dataset_filepath = config_reader.get_dataset_path(
    level=config_reader.preprocessed_files_dirname, 
    filename=config_reader.get("prompt_preprocessed_dirname"), 
)
dataset_dict.save_to_disk(preprocessed_prompt_dataset_filepath)

Saving the dataset (1/1 shards): 100%|██████████| 44/44 [00:00<00:00, 5059.47 examples/s]


## Combinação do dataset de redações com o de temas

In [18]:
essay_dataset_dict = load_from_disk(preprocessed_essay_dataset_filepath)
essay_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 459
    })
    test: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 98
    })
    validation: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 99
    })
})

In [19]:
prompt_dataset = load_from_disk(preprocessed_prompt_dataset_filepath)["train"]
prompt_dataset

Dataset({
    features: ['topic_id', 'topic_text'],
    num_rows: 44
})

In [20]:
combinator = EssayAndPromptDatasetsCombinator()
combinator

In [21]:
combined_dataset_dict = combinator.combine(essay_dataset_dict, prompt_dataset)
combined_dataset_dict

DatasetDict({
    test: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 98
    })
    train: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 459
    })
    validation: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 99
    })
})

In [22]:
# Save processed dataset for later use:
processed_dataset_filepath = config_reader.get_dataset_path(
    level=config_reader.processed_files_dirname, 
    filename=config_reader.get("processed_dirname"), 
)
combined_dataset_dict.save_to_disk(processed_dataset_filepath)

Saving the dataset (1/1 shards): 100%|██████████| 99/99 [00:00<00:00, 7677.19 examples/s] 


## Exemplo da tokenização de uma redação

In [23]:
essay_dataset_dict = load_from_disk(preprocessed_essay_dataset_filepath)
essay_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 459
    })
    test: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 98
    })
    validation: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 99
    })
})

In [24]:
essay_train_dataset = essay_dataset_dict["train"]
essay_examples = essay_train_dataset.shuffle(23).select(range(3))["essay_text"]
for example in essay_examples:
    print(example)

O costumeiro descaso dos governos, independentemente de esferas ou de partidos, é o responsável pelo surgimento e expansão das cracolândias de São Paulo e de outras capitais do Brasil. Não só não se fez nada para combater o problema em suas origens, como se foi, posteriormente, “empurrando com a barriga”, provavelmente porque as diversas administrações paulistas e paulistanas acreditavam que aquela área decadente da cidade jamais viria a prejudicá-los na hora de pedir votos para se eleger. Houve até prefeito que tentou se capitalizar politicamente com aquela versão tupiniquim do inferno, acolhendo de braços abertos os dependentes de crack e até lhes dando uns trocados para sustentarem o pobre do vício, que viria da exclusão social e demais maldades intrínsecas ao capitalismo.Desse modo, a cracolândia, que já ia de vento em popa sem a colaboração municipal, tornou-se ainda maior e se transformou numa espécie de Galeria Pajé dos psicotrópicos, reunindo consumidores e fornecedores a céu a

In [25]:
tokenizer = AutoTokenizer.from_pretrained(config_reader.get("checkpoint"), do_lower_case=False)
tokenizer

BertTokenizerFast(name_or_path='neuralmind/bert-base-portuguese-cased', vocab_size=29794, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [26]:
tokenized_essays = []
for essay in essay_examples:
    tokenized_essay = tokenizer.tokenize([essay], is_split_into_words=True)
    tokenized_essays.append(tokenized_essay)
tokenized_essays[0][:20]

['O',
 'costume',
 '##iro',
 'des',
 '##caso',
 'dos',
 'governos',
 ',',
 'independente',
 '##mente',
 'de',
 'esferas',
 'ou',
 'de',
 'partidos',
 ',',
 'é',
 'o',
 'responsável',
 'pelo']

## Fine-tuning para classificar nota em relação à abordagem do tema da redação

In [27]:
dataset_dict = load_from_disk(processed_dataset_filepath)
dataset_dict

DatasetDict({
    test: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 98
    })
    train: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 459
    })
    validation: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 99
    })
})

In [28]:
fine_tuner = TopicDeviationFineTuner(config_reader.get("checkpoint"), dataset_dict)
fine_tuner

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/mateuslatrova/anaconda3/envs/enem/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Map: 100%|██████████| 99/99 [00:00<00:00, 492.73 examples/s]


In [29]:
fine_tuner.run_model_training()

  0%|          | 0/174 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# Test:
# Como escolher o melhor modelo?
fine_tuner.run_model_test()

{'accuracy': 0.5918367346938775, 'f1': 0.7101449275362319}